In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings

warnings.filterwarnings('ignore')
np.random.seed(1)

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [48]:
X_train=pd.read_csv('train.csv')
y_train=pd.read_csv('train_.csv')
X_test=pd.read_csv('test.csv')

In [35]:
y_train=y_train['0.1']

In [36]:
y_train.head()

0    1
1    1
2    1
3    0
4    0
Name: 0.1, dtype: int64

In [37]:
y_train.loc[-1] = 0  # adding a row
y_train.index = y_train.index + 1  # shifting index
y_train = y_train.sort_index()  # sorting by index

In [38]:
dtrain = xgb.DMatrix(X_train.values,y_train.values)
dtest = xgb.DMatrix(X_test.values)
param = {'silent': 1, 'objective':'binary:logistic','max_depth': 7,'num_round':30}


In [39]:
bst= xgb.train(param, dtrain)

In [40]:


y_pred = bst.predict(dtrain, pred_leaf=True)
num_leaf= 1000
X_train_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
for i in range(0,len(y_pred)):
    temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
    X_train_leaves[i][temp] += 1

y_pred= bst.predict(dtest, pred_leaf=True)    
X_test_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
for i in range(0,len(y_pred)):
    temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
    X_test_leaves[i][temp] += 1


bestC=0
auc_best=0


lm = LogisticRegression(penalty='l2',C=0.1) # logestic model construction
lm.fit(X_train_leaves,y_train)  # fitting the data

y_pred_est = lm.predict(X_test_leaves)   # Give the probabilty on each label


#---------------------------------------------------------------------------------------------



IndexError: too many indices for array

In [44]:
y_pred_est.shape

(418,)

In [45]:
test_df=pd.read_csv('test.csv')

In [47]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': y_pred_est
})
submission.to_csv('titanic.csv', index=False)